In [6]:
%pwd

'c:\\Users\\Viswajith13\\Desktop\\CHATBOT\\CollegeFAQ-Chatbot\\research'

In [7]:
import os
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\Viswajith13\\Desktop\\CHATBOT\\CollegeFAQ-Chatbot'

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
#Loading the pdf files
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [11]:
extracted_data=load_pdf_file(data='Data/')

In [12]:
#extracted_data

In [13]:
#Splitting Data to Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 12


In [12]:
#text_chunks

[Document(metadata={'source': 'Data\\College_book.pdf', 'page': 0}, page_content='Vishwa School of Engineering - Information Handbook \nAdmission Procedure \n \n1. Check Eligibility: Candidates must meet the required academic qualifications, including a \nminimum of 60% in 12th grade (Science stream) with Physics, Chemistry, and Mathematics. \n2. Entrance Exam: Appear for the Vishwa Engineering Entrance Test (VEET) or provide \nvalid scores for JEE Mains. The VEET is conducted twice a year, with detailed syllabi \navailable on our website.'),
 Document(metadata={'source': 'Data\\College_book.pdf', 'page': 0}, page_content='3. Application Process: Submit the online application through our user-friendly portal. \nTrack your application status in real time. \n4. Counseling and Seat Allotment: Participate in a seamless counseling process, with options \nto choose preferred branches based on merit. \n5. Fee Payment and Document Verification: Pay fees online through secure gateways and \ncom

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
#Function to download embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\Viswajith13\AppData\Local\Temp\ipykernel_20192\3440179883.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Viswajith13\anaconda3\envs\clgbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
#query_result

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "clgbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
# Embedding each chunk and uploading them to pinecone database for using as knowledgebase for Models
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [25]:
# Loading the index stored in pinecone to project 

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("Where is the campus located?")

In [29]:
retrieved_docs

[Document(id='c8e2fe4e-9ac0-4e77-baa5-4a42aa34c9e2', metadata={'page': 0.0, 'source': 'Data\\College_book.pdf'}, page_content='Situated in the educational hub of Tamil Nadu, this campus combines academic excellence \nwith cultural richness.  \n   It is well-connected by metro and rail services and located only 10 km from Chennai \nInternational Airport.  \n   The campus hosts state-of-the-art labs, libraries, and a vibrant student community.'),
 Document(id='e9045d41-c207-49c4-89a4-461b273b3b54', metadata={'page': 1.0, 'source': 'Data\\College_book.pdf'}, page_content='It is located 25 km from Vijayawada Airport and features cutting-edge facilities for \nacademics and research. \n \n6. New Delhi: \n   The campus is located in the vibrant capital of India, offering students excellent exposure \nto industry leaders and government institutions.  \n   It is well-connected by metro and road, with proximity to major landmarks. The New Delhi \ncampus emphasizes collaboration with top-tier org

In [1]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [2]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [3]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'retriever' is not defined

In [91]:
response = rag_chain.invoke({"input": "What are the admission criteria?"})
print(response["answer"])



The admission criteria for Vishwa School of Engineering include meeting the required academic qualifications, appearing for the Vishwa Engineering Entrance Test (VEET) or providing valid scores for JEE Mains, and paying the required fees. Scholarships are also available for top performers, economically weaker students, and state and national-level athletes.
